In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# -------------------- Configuration Parameters --------------------
VIDEO_PATH = "jump_video.mp4"  # Replace with your video file path

# Detection thresholds: adjusted for better sensitivity
TAKEOFF_THRESHOLD = -0.02     # Negative change indicates upward movement (takeoff)
LANDING_THRESHOLD = 0.02     # Positive change indicates downward movement (landing)

# Biomechanical constants
G = 9.81                      # Gravity (m/s^2)
MASS = 70                     # Athlete's mass in kg

# Height conversion factor (pixels to meters)
# This is a crucial parameter that needs calibration based on your video
# Typically done by measuring a known object in the video
HEIGHT_CONVERSION_FACTOR = 1.0  # Adjust this based on your video calibration

# -------------------- Initialization --------------------
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Create pose estimator with higher accuracy settings
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,  # Use the most accurate model
    smooth_landmarks=True,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Open video capture
cap = cv2.VideoCapture(VIDEO_PATH)
FPS = cap.get(cv2.CAP_PROP_FPS)
DT = 1 / FPS if FPS > 0 else 0.033  # Default to ~30 FPS if undetectable

# Data storage containers
hip_positions = []    # Store averaged hip y positions per frame
knee_angles = []      # Store averaged knee angles per frame
time_stamps = []      # Timestamp for each frame
jump_data = []        # Final jump metrics will be stored here

# Lists for storing takeoff and landing events
takeoff_indices = []
landing_indices = []

# Variables for jump detection
frame_count = 0
prev_hip_y = None
jumping = False  # Flag indicating whether the athlete is mid-air
hip_velocity_buffer = []  # Buffer to smooth velocity calculations

# -------------------- Function Definitions --------------------
def calculate_angle(a, b, c):
    """
    Calculate the angle (in degrees) at point b formed by the points a, b, and c.
    """
    ba = np.array([a.x - b.x, a.y - b.y])
    bc = np.array([c.x - b.x, c.y - b.y])
    
    # Add extra checks for valid vectors
    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)
    
    if norm_ba < 1e-6 or norm_bc < 1e-6:
        return 0.0  # Return 0 angle if vectors are too small
        
    cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0)) * (180 / np.pi)
    return np.clip(angle, 0, 180)

def extract_features(landmarks):
    """
    From the pose landmarks extract:
      - Averaged hip vertical position (hip_y)
      - Knee flexion angles (both individual and averaged)
    """
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
    right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
    
    # Averaged vertical position of the hips (note: lower value means higher on the image)
    hip_y = (left_hip.y + right_hip.y) / 2

    # Compute knee angles using the cosine formula
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    avg_knee_angle = (left_knee_angle + right_knee_angle) / 2
    
    return hip_y, avg_knee_angle, left_knee_angle, right_knee_angle

def smooth_velocity(hip_y, velocity_buffer, buffer_size=5):
    """
    Apply moving average smoothing to velocity calculations to reduce noise
    """
    velocity_buffer.append(hip_y)
    if len(velocity_buffer) > buffer_size:
        velocity_buffer.pop(0)
    
    if len(velocity_buffer) >= 2:
        # Calculate smoothed velocity using the last few frames
        smoothed_velocity = velocity_buffer[-1] - velocity_buffer[-2]
        return smoothed_velocity
    return 0

def compute_jump_height(takeoff_idx, peak_idx, hip_positions):
    """
    Compute the actual jump height using improved methodology.
    This uses the difference between standing height at takeoff and
    the peak height during flight, converted to meters.
    """
    takeoff_hip = hip_positions[takeoff_idx]
    peak_hip = hip_positions[peak_idx]
    
    # In image coordinates, lower y values are higher positions (top of screen)
    hip_difference = takeoff_hip - peak_hip  
    
    # Convert from normalized coordinates to actual height in meters
    jump_height_meters = hip_difference * HEIGHT_CONVERSION_FACTOR
    
    return max(0, jump_height_meters)  # Ensure non-negative height

def compute_jump_features(takeoff_idx, landing_idx, hip_positions, knee_angles, time_stamps):
    """
    For a jump defined by its takeoff and landing frame indices:
      - Compute jump height using improved methodology
      - Compute flight time as the elapsed time between takeoff and landing
      - Compute RSImod = jump_height / flight_time
    """
    # Find the peak (apex) of the jump - minimum hip y-value between takeoff and landing
    jump_segment = hip_positions[takeoff_idx:landing_idx+1]
    if len(jump_segment) > 0:
        peak_offset = np.argmin(jump_segment)
        peak_idx = takeoff_idx + peak_offset
    else:
        peak_idx = takeoff_idx
    
    # Compute jump height with proper conversion
    jump_height = compute_jump_height(takeoff_idx, peak_idx, hip_positions)
        
    # Flight time between takeoff and landing
    flight_time = time_stamps[landing_idx] - time_stamps[takeoff_idx]
    
    # RSImod calculation - this should match your expected values with proper calibration
    rsimod = jump_height / flight_time if flight_time > 0 else 0

    # Calculate knee angles for symmetry analysis
    # Get knee angles at takeoff position
    takeoff_knee_angle = knee_angles[takeoff_idx] if takeoff_idx < len(knee_angles) else 0
    # Get minimum knee angle during eccentric phase
    eccentric_knee_angles = knee_angles[takeoff_idx:peak_idx+1]
    min_knee_angle = min(eccentric_knee_angles) if eccentric_knee_angles else takeoff_knee_angle
    
    # Phase durations
    eccentric_duration = time_stamps[peak_idx] - time_stamps[takeoff_idx]
    concentric_duration = time_stamps[landing_idx] - time_stamps[peak_idx]
    
    # Calculate average velocity during takeoff
    velocity_values = np.diff(hip_positions[takeoff_idx:peak_idx+1]) / DT if peak_idx > takeoff_idx else [0]
    max_velocity = np.min(velocity_values) * -1 if len(velocity_values) > 0 else 0  # Convert to positive value
    
    # Calculate power output based on force (mass * gravity) and velocity
    power_output = MASS * G * jump_height / flight_time if flight_time > 0 else 0
    
    return {
        "Jump": None,  # To be filled in later
        "Jump Height (m)": round(jump_height, 3),
        "Flight Time (s)": round(flight_time, 3),
        "RSImod": round(rsimod, 3),
        "Peak Knee Angle (deg)": round(min_knee_angle, 1),
        "Eccentric Duration (s)": round(eccentric_duration, 3),
        "Concentric Duration (s)": round(concentric_duration, 3),
        "Max Velocity (m/s)": round(max_velocity, 3),
        "Power Output (W)": round(power_output, 1)
    }

# -------------------- Video Processing Loop --------------------
prev_velocity = 0
velocity_smoothing_factor = 0.5  # Lower values = more smoothing

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    time_stamps.append(frame_count * DT)
    
    # Convert frame to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Draw landmarks for visualization
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract features for current frame
        hip_y, avg_knee_angle, left_knee_angle, right_knee_angle = extract_features(landmarks)
        hip_positions.append(hip_y)
        knee_angles.append(avg_knee_angle)

        # Jump event detection using improved velocity calculation with smoothing
        if prev_hip_y is not None:
            raw_velocity = hip_y - prev_hip_y
            # Apply exponential smoothing to velocity
            velocity = raw_velocity * velocity_smoothing_factor + prev_velocity * (1 - velocity_smoothing_factor)
            
            # State machine for jump detection
            if velocity < TAKEOFF_THRESHOLD and not jumping:
                # Detected takeoff - verify with a few checks
                takeoff_indices.append(frame_count)
                jumping = True
                print(f"Takeoff detected at frame {frame_count}, velocity: {velocity:.4f}")
                
            elif velocity > LANDING_THRESHOLD and jumping:
                # Detected landing
                landing_indices.append(frame_count)
                jumping = False
                print(f"Landing detected at frame {frame_count}, velocity: {velocity:.4f}")
            
            prev_velocity = velocity

        prev_hip_y = hip_y  # Update for next frame

        # Display real-time info
        cv2.putText(frame, f"Jumps: {len(takeoff_indices)}", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f"Knee Angle: {int(avg_knee_angle)}", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        if jumping:
            cv2.putText(frame, "JUMPING", (50, 150),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    cv2.imshow('Jump Analysis', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# -------------------- Post-Processing: Compute Jump Statistics --------------------
# Match takeoff and landing events
print(f"\nDetected {len(takeoff_indices)} takeoffs and {len(landing_indices)} landings")

# Ensure correct pairing of takeoff and landing events
valid_jumps = []
for i, t_idx in enumerate(takeoff_indices):
    # Find the next landing event after this takeoff
    next_landings = [l for l in landing_indices if l > t_idx]
    if next_landings:
        valid_jumps.append((t_idx, min(next_landings)))

# Compute metrics for valid jumps
for i, (t_idx, l_idx) in enumerate(valid_jumps):
    # Check if indices are within data range
    if t_idx >= len(hip_positions) or l_idx >= len(hip_positions):
        continue
    
    features = compute_jump_features(t_idx, l_idx, hip_positions, knee_angles, time_stamps)
    features["Jump"] = i + 1
    jump_data.append(features)

# Print the jump metrics including RSImod
print("\n===== JUMP ANALYSIS RESULTS =====")
for data in jump_data:
    print(f"\nJump #{data['Jump']}:")
    for key, value in data.items():
        if key != "Jump":
            print(f"  {key}: {value}")

# Save results to CSV
try:
    import pandas as pd
    df = pd.DataFrame(jump_data)
    df.to_csv("jump_analysis_results.csv", index=False)
    print("\nResults saved to jump_analysis_results.csv")
except ImportError:
    print("\nPandas not installed. Results not saved to CSV.")
